<a href="https://colab.research.google.com/github/MJ-best/-DACON-AI-/blob/main/%5Bfeature_Calc%5Dminjun_plasma_diagnosis_with_ML_20230225.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electrcial Diagnosis for Dielectric Barrier Discharge(DBD) using Measurement Data driven Machine Learning Technique
- e-mail : minjoun@gmail.com
- SPDX-FileCopyrightText: © 2023 KIM MINJUN  minjoun@gmail.com
- SPDX-License-Identifier: BSD-3-Clause

BSD 3-Clause License

Copyright © 2023 KIM MINJUN minjoun@gmail.com

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

● Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

● Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

● Neither the name of the KIM MINJUN nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
import datetime
from pytz import timezone

print("last update : ",datetime.datetime.now(timezone('Asia/Seoul')))

last update :  2023-02-25 23:05:36.290511+09:00


In [2]:
import io
import os
import numpy as np
import pandas as pd
import math
from scipy import optimize
from scipy.signal import find_peaks, peak_prominences, chirp, peak_widths
import matplotlib.pyplot as plt

In [ ]:
def plasma_calc(data):
    info = pd.read_csv(data, encoding='utf-8-sig', error_bad_lines=False).reset_index()
    info.columns = ["V_Time" , "Voltage", "C_Time", "Current", "None"]
    del info["None"]
    # Bring Date from oscilloscope
    Date = pd.to_datetime(info["Voltage"][14])
    # Bring Sampling Period from oscilloscope
    Sampling_Period = float(info["Voltage"][12])
    # Calculating experimental Sampling Period by substracting two measured time infomations 
    dt = float(info["V_Time"][18]) - float(info["V_Time"][17])
    # Bring Voltage Probe's Ratio and Current Probe's Infomation
    V_Probe_Ratio = float(info["Voltage"][4])
    C_Probe_Ratio = float(info["Current"][4])
    IV = info[['Voltage', 'Current']].copy()
    IV = IV.drop(list(range(0,17)))
    IV = IV.reset_index(drop = True)
    IV = IV.astype('float')

    I = IV['Current'].to_numpy()
    I = np.round(I, 6)
    V = IV['Voltage'].to_numpy()
    V = np.round(V,1)

    zero_idx = np.where(V == 0)
    diff_zero_idx = np.diff(zero_idx)
    zero_point_idx = np.where(diff_zero_idx > 100) 
    zero_point_idx = list(zero_point_idx[1])

    for i in range(0,len(zero_idx[0][zero_point_idx])-2):

        t1 = zero_idx[0][zero_point_idx][i]
        t2 = zero_idx[0][zero_point_idx][i+2]

        I_dc = np.mean(I[t1:t2]) #Current DC offset

        I_1cyc = I[t1:t2] - I_dc
        I_1cyc = np.round(I_1cyc,6)
        V_1cyc = V[t1:t2]
        V_1cyc = np.round(V_1cyc,6)

        T_1cyc = np.array(list(range(0,len(I_1cyc))))*dt
        T_N = len(I_1cyc)
        Period = len(I_1cyc)*dt
        Freq = 1/Period

        VV = np.zeros(T_N)
        VV_int = [0.0]

        VV = np.array([V_1cyc[j] * V_1cyc[j] for j in range(T_N)])
        for j in range(T_N):
            VV_int = VV_int + VV[j]*dt

        Vrms = math.sqrt(VV_int/Period)

        II = np.zeros(T_N)
        II_int = [0.0]

        II = np.array([I_1cyc[j] * I_1cyc[j] for j in range(T_N)])
        for j in range(T_N):
            II_int = II_int + II[j]*dt

        Irms = math.sqrt(II_int/Period)

        Energy_1cyc = np.zeros(T_N)
        Energy_int = [0.0]

        Energy_1cyc = np.array([V_1cyc[j]*I_1cyc[j] for j in range(T_N)])
        for j in range(T_N):
            Energy_int = Energy_int + Energy_1cyc[j]*dt

        Energy = Energy_int[0]

        Q_1cyc = np.zeros(T_N)
        Q_int = [0.0]

        for j in range(T_N):
            Q_int = Q_int + I_1cyc[j]*dt
            Q_1cyc[j] = Q_int

        Q_1cyc = np.round(Q_1cyc, 12)

    def find_peak_and_width(data, w):
      peaks, _ = find_peaks(data, width = 2)
      prominences = peak_prominences(data, peaks)[0]
      constant = w # constant depeond on peak dection sensitivity (1.001 ~ 10) 
      prominence_new = np.percentile(prominences, [0, 25, 50, 75, 100])[2]*constant
      new_peaks, _ = find_peaks(data, prominence = prominence_new)
      new_prominences = peak_prominences(data, new_peaks)[0]
      new_widths = peak_widths(data, new_peaks, rel_height = 0.5)

      def find_outlier(data):
        q1, q3 = np.percentile(data, [25, 75])
        iqr = q3 - q1
        lower_bound = q1 - (iqr * 1.5) # Q1 - (iqr * 1.5)
        upper_bound = q3 + (iqr * 1.5) # Q3 + (iqr * 1.5)
        outlier = np.where((data > upper_bound) | (data < lower_bound)) # 최대값보다 크고, 최소값보다 작은 데이터의 인덱스 추출
        return outlier

      final_peaks = np.delete(new_peaks, find_outlier(new_widths[0]))
      final_widths = peak_widths(data, final_peaks, rel_height = 0.5) # rel_height = 0.5 is same definition in MATLAB
      return final_peaks, final_widths[0]

    #플라즈마 방전구간 설정
    #방전구간 T0-T1, T2-T3

    IV_1cyc = I_1cyc * V_1cyc
    peaks, widths = find_peak_and_width(IV_1cyc, 1.5)
    diff_peak_idx = np.diff(peaks)
    plasma_off_idx = np.argmax(diff_peak_idx)
    off_idx = plasma_off_idx.tolist()

    T0 = peaks[0]
    T1 = peaks[off_idx]
    T2 = peaks[np.argmax(diff_peak_idx)+1]
    T3 = peaks[-1]
    T4 = len(V_1cyc)

    # V의 파형이 싸인인지 역싸인인지를 구분한다.

    if np.argmin(V_1cyc) > np.argmax(V_1cyc):
    #sine 
      sig_sine = True
    else: 
    #-sine
      sig_sine = False 

    #방전이 Vmax 시점 보다 빨리 끝나는 경우 T1과 T3 시점을 수정해야한다 (파형도고려!)
    if sig_sine == True:
        if V_1cyc[T1] >= np.max(V_1cyc):
          T1 = T1
        else:
          T1 = np.argmax(V_1cyc)
    else:
        if V_1cyc[T1] <= np.min(V_1cyc):
          T1 = T1
        else:
          T1 = np.argmin(V_1cyc)

    if sig_sine == True:
        if V_1cyc[T3] <= np.min(V_1cyc):
          T3 = T3
        else:
          T3 = np.argmin(V_1cyc)
    else:
        if V_1cyc[T3] >= np.max(V_1cyc):
          T3 = T3
        else:
          T3 = np.argmax(V_1cyc)

    Vmin_1 = np.max(V_1cyc[peaks][V_1cyc[peaks]<0])
    Vmin_2 = np.min(V_1cyc[peaks][V_1cyc[peaks]>0])
    Vmax_1 = V_1cyc[T1]
    Vmax_2 = V_1cyc[T3]

    if sig_sine :
        I_1 = np.array(-I_1cyc[T2:T4])
        I_2 = np.delete(I_1cyc, range(T2,T4))
        I_3 = np.insert(I_2, T2, I_1)
        I_1cyc_inv = I_3
    else:
        I_1 = np.array(-I_1cyc[T0:T2])
        I_2 = np.delete(I_1cyc, range(T0,T2))
        I_3 = np.insert(I_2, T0, I_1)
        I_1cyc_inv = I_3

    if sig_sine :
        V_1 = np.array(-V_1cyc[T2:T4])
        V_2 = np.delete(V_1cyc, range(T2,T4))
        V_3 = np.insert(V_2, T2, V_1)
        V_1cyc_inv = V_3
    else:
        V_1 = np.array(-V_1cyc[T0:T2])
        V_2 = np.delete(V_1cyc, range(T0,T2))
        V_3 = np.insert(V_2, T0, V_1)
        V_1cyc_inv = V_3

    #T0:T1 첫 반주기에서의 방전구간 내의 피크갯수를 계산
    first_period_I_cyc_inv= I_1cyc_inv[T0:T1]
    first_period_peaks, first_period_widths = find_peak_and_width(first_period_I_cyc_inv, 2)

    #number of dischrages per period
    first_period_num_discharges = len(first_period_peaks)
    #each current pulse duration data
    first_period_pulse_duration = first_period_widths*dt
    first_period_pulse_duration_mean = np.mean(first_period_pulse_duration)
    first_period_pulse_duration_std = np.std(first_period_pulse_duration)
    #each current pulse energy data
    #peak_energy = widths[0]*dt*abs(I_1cyc[peaks])*abs(V_1cyc[peaks])
    first_period_peak_energy = first_period_widths*dt*I_1cyc_inv[first_period_peaks]*V_1cyc_inv[first_period_peaks]
    first_period_peak_energy_mean = np.mean(first_period_peak_energy)
    first_period_peak_energy_std = np.std(first_period_peak_energy)

    second_period_I_cyc_inv= I_1cyc_inv[T2:T3]
    second_period_peaks, second_period_widths = find_peak_and_width(second_period_I_cyc_inv, 2)

    #number of dischrages per period
    second_period_num_discharges = len(second_period_peaks)
    #each current pulse duration data
    second_period_pulse_duration = second_period_widths*dt
    second_period_pulse_duration_mean = np.mean(second_period_pulse_duration)
    second_period_pulse_duration_std = np.std(second_period_pulse_duration)
    #each current pulse energy data
    #peak_energy = widths[0]*dt*abs(I_1cyc[peaks])*abs(V_1cyc[peaks])
    second_period_peak_energy = second_period_widths*dt*I_1cyc_inv[second_period_peaks]*V_1cyc_inv[second_period_peaks]
    second_period_peak_energy_mean = np.mean(second_period_peak_energy)
    second_period_peak_energy_std = np.std(second_period_peak_energy)

    def linear_fit(x, a, b):
      return a*x+b

    #T0:T1 plasma on : dielectric capacitance
    popt0, pcov0 = optimize.curve_fit(linear_fit, V_1cyc[T0:T1], Q_1cyc[T0:T1])
    Zdiel_1 = popt0[0]
    Zdiel_1_error = np.sqrt(np.diag(pcov0))[0]
    #T1:T2 plasma off : dielectric + air gap capacitance
    popt1, pcov1 = optimize.curve_fit(linear_fit, V_1cyc[T1:T2], Q_1cyc[T1:T2])
    Ctot_1 = popt1[0]
    Ctot_1_error = np.sqrt(np.diag(pcov1))[0]
    #T2:T3 plasma on : dielectric capacitance
    popt2, pcov2 = optimize.curve_fit(linear_fit, V_1cyc[T2:T3], Q_1cyc[T2:T3])
    Zdiel_2 = popt2[0]
    Zdiel_2_error = np.sqrt(np.diag(pcov2))[0]
    #T3:T4 plasma off : dielectric + air gap capacitance
    popt3, pcov3 = optimize.curve_fit(linear_fit, V_1cyc[T3:T4], Q_1cyc[T3:T4])
    Ctot_2 = popt3[0]
    Ctot_2_error = np.sqrt(np.diag(pcov3))[0]
    '''
    #Qmax = Zdiel * del V 식으로 계산
    Qmax_1 = Zdiel_1* abs(Vmax_1 - Vmin_1)
    Qmax_2 = Zdiel_2* abs(Vmax_2 - Vmin_2)
     ''' 
    discharge_time_list_1 = list(range(T0,T1))
    discharge_time_list_2 = list(range(T2,T3))

    Q_int_1 = [0.0]
    Qt_1 = np.zeros(discharge_time_list_1[-1])
    for j in discharge_time_list_1:
        Q_int_1 = Q_int_1 + I_1cyc_inv[j-1]*dt
        Qt_1[j-1] = Q_int_1

    Q_int_2 = [0.0]
    Qt_2 = np.zeros(discharge_time_list_2[-1])
    for k in discharge_time_list_2:
        Q_int_2 = Q_int_2 + I_1cyc_inv[k-1]*dt
        Qt_2[k-1] = Q_int_2

    Q_transfer_1 = np.delete(Qt_1, np.where(Qt_1 == 0.0))
    Qmax_1 = Q_transfer_1[-1]
    Q_transfer_2 = np.delete(Qt_2, np.where(Qt_2 == 0.0))
    Qmax_2 = Q_transfer_2[-1]

    delta_Q1 =abs(np.max(Q_1cyc[first_period_peaks])-np.min(Q_1cyc[first_period_peaks]))
    delta_Q2 =abs(np.max(Q_1cyc[second_period_peaks])-np.min(Q_1cyc[second_period_peaks]))

    Cdiel_1 = abs(Ctot_1 / (1-(Qmax_1 / delta_Q1)+((Ctot_1/Zdiel_1)*(Qmax_1/delta_Q1))))
    Cdiel_2 = abs(Ctot_2 / (1-(Qmax_2 / delta_Q2)+((Ctot_2/Zdiel_2)*(Qmax_2/delta_Q2))))

    #1/Ctot = 1/Cdiel + 1/Cgas
    Cgas_1 = (Cdiel_1 * Ctot_1)/(Cdiel_1 - Ctot_1)
    Cgas_2 = (Cdiel_2 * Ctot_2)/(Cdiel_2 - Ctot_2)

    # Using Capacitance calcuate energy by Modified Manlay Method
    Power_manely_1 = 4*Cdiel_1*Vmin_1*(Vmax_1-Vmin_1)/(1+Cgas_1/Cdiel_1)
    Power_manely_2 = 4*Cdiel_2*Vmin_2*(Vmax_2-Vmin_2)/(1+Cgas_2/Cdiel_2)

    #Power factor
    #Manley의 방법에 의해 계산된 값은 일 즉 Power [W] 이고 전류와 전압의 곱을 시간으로 나눠계산한 Energy [VA]는 피상전력이므로 이 둘을 이용해 power factor를 계산할 수 있다.
    power_factor_1 = Power_manely_1 / Energy
    power_factor_2 = Power_manely_2 / Energy
    #theta_1 = math.acos(power_factor_1)
    #theta_2 = math.acos(power_factor_2)

    #R Valdiva Barrientos의 경험식 V break down voltage
    Vbd_1 = (1.4*(Cdiel_1/Cgas_1)**2)/ np.log(Freq)
    Vbd_2 = (1.4*(Cdiel_2/Cgas_2)**2)/ np.log(Freq)

    a1 = (Cdiel_1 - Zdiel_1)/(Cdiel_1 - Ctot_1)*100
    a2 = (Cdiel_2 - Zdiel_2)/(Cdiel_2 - Ctot_2)*100
    b1 = (Zdiel_1 - Ctot_1)/(Cdiel_1 - Ctot_1)*100
    b2 = (Zdiel_2 - Ctot_2)/(Cdiel_2 - Ctot_2)*100

    #하나의 펄스가 커패시턴스의 충방전과 같이 생각한다면 2RCd를 이용해서 플라즈마 저항값을 계산할 수 있다
    plasma_resistance_1 = first_period_pulse_duration / (2*Cdiel_1)
    plasma_resistance_2 = second_period_pulse_duration / (2*Cdiel_2)
    plasma_resistance_1_mean = np.mean(plasma_resistance_1)
    plasma_resistance_1_std = np.std(plasma_resistance_1)
    plasma_resistance_2_mean = np.mean(plasma_resistance_2)
    plasma_resistance_2_std = np.std(plasma_resistance_2)

    # Electon Density Calculation ne(/cm^3) first half period
    distance_cm = distance * 100
    electron_mobility = 592.12
    electron_charge = 1.602176634e-19

    E_1 = abs(V_1cyc[first_period_peaks])/distance_cm
    A = distance_cm * distance_cm * math.pi
    J_1 =abs(I_1cyc[first_period_peaks]/A)

    first_period_peak_electron_density = J_1/(E_1*electron_mobility*electron_charge)
    first_period_peak_electron_density_mean = np.mean(first_period_peak_electron_density)
    first_period_peak_electron_density_std = np.std(first_period_peak_electron_density)

    # Electon Density Calculation ne(/cm^3) second half period
    E_2 = abs(V_1cyc[second_period_peaks])/distance_cm
    A = distance_cm * distance_cm * math.pi
    J_2 = abs(I_1cyc[second_period_peaks]/A)

    second_period_peak_electron_density = J_2/(E_2*electron_mobility*electron_charge)
    second_period_peak_electron_density_mean = np.mean(second_period_peak_electron_density)
    second_period_peak_electron_density_std = np.std(second_period_peak_electron_density)

    #Energy density required to sustain a discharge
    Wion = 50 #effective ionization energy in atmospheric air
    t_electron = 10E-9 #average lifetime of electron in atmoshperic air 
    Psustain1 = first_period_peak_electron_density_mean * Wion / t_electron
    Psustain2 = second_period_peak_electron_density_mean*Wion / t_electron

    if sig_sine == True:
      positive_period_num_discharges = first_period_num_discharges
      positive_period_peak_electron_density_mean=first_period_peak_electron_density_mean
      positive_period_peak_electron_density_std=first_period_peak_electron_density_std
      positive_period_peak_energy_mean=first_period_peak_energy_mean
      positive_period_peak_energy_std=first_period_peak_energy_std
      positive_period_pulse_duration_mean=first_period_pulse_duration_mean
      positive_period_pulse_duration_std=first_period_pulse_duration_std
      Zdiel_p = Zdiel_1
      Cdiel_p=Cdiel_1
      Cgas_p=Cgas_1
      Ctot_p=Ctot_1
      positive_period_plasma_resistance_mean = plasma_resistance_1_mean
      positive_period_plasma_resistance_std = plasma_resistance_1_std
      positive_period_power_manely = Power_manely_1
      positive_period_power_factor = power_factor_1
      #positive_period_phase_difference = theta_1
      positive_period_plasma_sustain_power=Psustain1
      positive_period_total_transfered_charge = Qmax_1
      positive_period_discharge_transfered_charge = delta_Q1
      Vbd_p = Vbd_1
      Vmin_p = Vmin_1
      positive_period_non_discharge_area = a1
      positive_period_discharge_area = b1

      negative_period_num_discharges = second_period_num_discharges
      negative_period_peak_electron_density_mean=second_period_peak_electron_density_mean
      negative_period_peak_electron_density_std=second_period_peak_electron_density_std
      negative_period_peak_energy_mean=second_period_peak_energy_mean
      negative_period_peak_energy_std=second_period_peak_energy_std
      negative_period_pulse_duration_mean=second_period_pulse_duration_mean
      negative_period_pulse_duration_std=second_period_pulse_duration_std
      Zdiel_n = Zdiel_2
      Cdiel_n=Cdiel_2
      Cgas_n=Cgas_2
      Ctot_n=Ctot_2
      negative_period_plasma_resistance_mean = plasma_resistance_2_mean
      negative_period_plasma_resistance_std = plasma_resistance_2_std
      negative_period_power_manely = Power_manely_2
      negative_period_power_factor = power_factor_2
      #negative_period_phase_difference = theta_2
      negative_period_plasma_sustain_power=Psustain2
      negative_period_total_transfered_charge = Qmax_2
      negative_period_discharge_transfered_charge = delta_Q2
      Vbd_n = -Vbd_2
      Vmin_n = Vmin_2
      negative_period_non_discharge_area = a2
      negative_period_discharge_area = b2

    else:
      positive_period_num_discharges = second_period_num_discharges
      positive_period_peak_electron_density_mean=second_period_peak_electron_density_mean
      positive_period_peak_electron_density_std=second_period_peak_electron_density_std
      positive_period_peak_energy_mean=second_period_peak_energy_mean
      positive_period_peak_energy_std=second_period_peak_energy_std
      positive_period_pulse_duration_mean=second_period_pulse_duration_mean
      positive_period_pulse_duration_std=second_period_pulse_duration_std
      Zdiel_p = Zdiel_2
      Cdiel_p=Cdiel_2
      Cgas_p=Cgas_2
      Ctot_p=Ctot_2
      positive_period_plasma_resistance_mean = plasma_resistance_2_mean
      positive_period_plasma_resistance_std = plasma_resistance_2_std
      positive_period_power_manely = Power_manely_2
      positive_period_power_factor = power_factor_2
      #positive_period_phase_difference = theta_2
      positive_period_plasma_sustain_power=Psustain2
      positive_period_total_transfered_charge = Qmax_2
      positive_period_discharge_transfered_charge = delta_Q2
      Vbd_p = Vbd_2
      Vmin_p = Vmin_2
      positive_period_non_discharge_area = a2
      positive_period_discharge_area = b2

      negative_period_num_discharges = first_period_num_discharges
      negative_period_peak_electron_density_mean=first_period_peak_electron_density_mean
      negative_period_peak_electron_density_std=first_period_peak_electron_density_std
      negative_period_peak_energy_mean=first_period_peak_energy_mean
      negative_period_peak_energy_std=first_period_peak_energy_std
      negative_period_pulse_duration_mean=first_period_pulse_duration_mean
      negative_period_pulse_duration_std=first_period_pulse_duration_std
      Zdiel_n = Zdiel_1
      Cdiel_n=Cdiel_1
      Cgas_n=Cgas_1
      Ctot_n=Ctot_1
      negative_period_plasma_resistance_mean = plasma_resistance_1_mean
      negative_period_plasma_resistance_std = plasma_resistance_1_std
      negative_period_power_manely = Power_manely_1
      negative_period_power_factor = power_factor_1
      #negative_period_phase_difference = theta_1
      negative_period_plasma_sustain_power=Psustain1
      negative_period_total_transfered_charge = Qmax_1
      negative_period_discharge_transfered_charge = delta_Q1
      Vbd_n = -Vbd_1
      Vmin_n = Vmin_1
      negative_period_non_discharge_area = a1
      negative_period_discharge_area = b1

    col_name = [
      "Measured Date",
      "Surface_degradation",
      "Gas",
      "Pressure[bar]",
      "Freqency[Hz]",
      "Energy[J]",
      "Vrms[V]",
      "Irms[A]",
      "gap distance[cm]",
      "positive_period_num_discharges[Num]",
      "positive_period_peak_electron_density_mean[/cm^3]",
      "positive_period_peak_electron_density_std",
      "positive_period_peak_energy_mean[J]",
      "positive_period_peak_energy_std",
      "positive_period_pulse_duration_mean[sec]",
      "positive_period_pulse_duration_std",
      "Zdiel_p[F]",
      "Cdiel_p[F]",
      "Cgas_p[F]",
      "Ctot_p[F]",
      "positive_period_plasma_resistance_mean[Ohm]",
      "positive_period_plasma_resistance_std",
      "positive_period_power_manely[W]",
      "positive_period_power_factor",
      #"positive_period_phase_difference_angle[degree]",
      "positive_period_plasma_sustain_energy_density[V/cm^3]",
      "positive_period_total_transfered_charge[C]",
      "positive_period_discharge_transfered_charge[C]",
      "Vbd_p[V]",
      "Vmin_p[V]",
      "positive_period_non_discharge_area[%]",
      "positive_period_discharge_area[%]",
      "negative_period_num_discharges[Num]",
      "negative_period_peak_electron_density_mean[/cm^3]",
      "negative_period_peak_electron_density_std",
      "negative_period_peak_energy_mean[J]",
      "negative_period_peak_energy_std",
      "negative_period_pulse_duration_mean[sec]",
      "negative_period_pulse_duration_std",
      "Zdiel_n[F]",
      "Cdiel_n[F]",
      "Cgas_n[F]",
      "Ctot_n[F]",
      "negative_period_plasma_resistance_mean[Ohm]",
      "negative_period_plasma_resistance_std",
      "negative_period_power_manely[W]",
      "negative_period_power_factor",
      #"negative_period_phase_difference[degree]",
      "negative_period_plasma_sustain_energy_density[V/cm^3]",
      "negative_period_total_transfered_charge[C]",
      "negative_period_discharge_transfered_charge[C]",
      "Vbd_n[V]",
      "Vmin_n[V]",
      "negative_period_non_discharge_area[%]",
      "negative_period_discharge_area[%]"
    ]

    result_data =[
      Date,
      surface_degradtion,
      gas,
      pressure,
      Freq,
      Energy,
      Vrms,
      Irms,
      distance_cm,
      positive_period_num_discharges,
      positive_period_peak_electron_density_mean,
      positive_period_peak_electron_density_std,
      positive_period_peak_energy_mean,
      positive_period_peak_energy_std,
      positive_period_pulse_duration_mean,
      positive_period_pulse_duration_std,
      Zdiel_p,
      Cdiel_p,
      Cgas_p,
      Ctot_p,
      positive_period_plasma_resistance_mean,
      positive_period_plasma_resistance_std,
      positive_period_power_manely,
      positive_period_power_factor,
      #positive_period_phase_difference,
      positive_period_plasma_sustain_power,
      positive_period_total_transfered_charge,
      positive_period_discharge_transfered_charge,
      Vbd_p,
      Vmin_p,
      positive_period_non_discharge_area,
      positive_period_discharge_area,
      negative_period_num_discharges,
      negative_period_peak_electron_density_mean,
      negative_period_peak_electron_density_std,
      negative_period_peak_energy_mean,
      negative_period_peak_energy_std,
      negative_period_pulse_duration_mean,
      negative_period_pulse_duration_std,
      Zdiel_n,
      Cdiel_n,
      Cgas_n,
      Ctot_n,
      negative_period_plasma_resistance_mean,
      negative_period_plasma_resistance_std,
      negative_period_power_manely,
      negative_period_power_factor,
      #negative_period_phase_difference,
      negative_period_plasma_sustain_power,
      negative_period_total_transfered_charge,
      negative_period_discharge_transfered_charge,
      Vbd_n,
      Vmin_n,
      negative_period_non_discharge_area,
      negative_period_discharge_area]

    Result = pd.DataFrame(result_data, index=col_name)
    Result = Result.transpose()
    return Result

# 데이터 분석을 위한 측정 데이터 준비
- CSV 파일을 직접 업로드 or 구글 드라이브에 연결해서 구글 드라이브 파일 경로 이용
- 측정데이터가 들어있는 폴더명을 입력하면 폴더 내부의 CSV파일을 자동으로 읽어온다
- 동일 조건으로 측정한 데이터를 한 폴더안에 넣어야 한다 (불러온 데이터를 한번에 계산하기 때문)

In [ ]:
#실험조건 입력 
gas = "air"
pressure = 1.013 # 1.013 bar (1 atm) 
distance = 100E-6 # 100um , 0.1mm
surface_degradtion = 0

In [ ]:
#측정데이터가 담긴 폴더에서 데이터분석을 자동화하는 코드블록
path = '/content/drive/MyDrive/*Plasma Code/plasma_data/m2_data/20200724 M2 40시간'

os.chdir(path)

file_list = os.listdir(path)
file_list.sort()
result = pd.DataFrame()

try:
    for i in range(len(file_list)-1):
        temp = plasma_calc(file_list[i])
        result = pd.concat([result,temp], ignore_index=True)
except:
    pass

In [ ]:
# 계산이후 데이터를 기존 데이터 베이스에 담을 때 사용
Result = pd.concat([Result,result],ignore_index=True)
Result.shape

In [ ]:
# 처음 데이터를 Result에 담을 때만 사용
Result = result

In [ ]:
# 데이터베이스 초기화할때만 사용
Result = pd.DataFrame()
result = pd.DataFrame()